## Monte Carlo Simulation to compare out of sample performance of strategies

First, we generate 10 series of random Gaussian returns (520 observations, equivalent to 2 years
of daily history), with 0 mean and an arbitrary standard deviation of 10%. Real prices exhibit
frequent jumps (Merton, 1976) and returns are not cross-sectionally independent, so we must add
random shocks and a random correlation structure to our generated data. Second, we compute
HRP, CLA and IVP portfolios by looking back at 260 observations (a year of daily history).
These portfolios are re-estimated and rebalanced every 22 observations (equivalent to a monthly
frequency). Third, we compute the out-of-sample returns associated with those three portfolios.
This procedure is repeated 10,000 times.

In intuitive terms, we can understand the above empirical results as follows: Shocks affecting a specific
investment penalize CLA’s concentration. Shocks involving several correlated investments
penalize IVP’s ignorance of the correlation structure. HRP provides better protection against
both, common and idiosyncratic shocks, by finding a compromise between diversification across
all investments and diversification across clusters of investments at multiple hierarchical levels.

In [6]:
import scipy.cluster.hierarchy as sch,random,numpy as np,pandas as pd,CLA
import matplotlib.pyplot as mpl
from HRP_Original import correlDist,getIVP,getQuasiDiag,getRecBipart,QPsol
import xlsxwriter
#------------------------------------------------------------------------------
def generateData(nObs,sLength,size0,size1,mu0,sigma0,sigma1F):
    # Time series of correlated variables
    #1) generate random uncorrelated data
    x=np.random.normal(mu0,sigma0,size=(nObs,size0)) # each row is a variable
    #2) create correlation between the variables
    cols=[random.randint(0,size0-1) for i in xrange(size1)]
    #print cols
    cols2 = [random.randint(0,size0-1) for i in xrange(size1)]
    cols3 = [random.randint(0,size0-1) for i in xrange(size1)]
    cols4 = [random.randint(0,size0-1) for i in xrange(size1)]
    #x = x + x[:,cols3]/4 + x[:,cols]/2+np.random.normal(0,.1,size=(nObs,len(cols)))
    #y=x[:,cols4]/8+x[:,cols3]/4+x[:,cols2]/2+x[:,cols]+np.random.normal(0,sigma0*sigma1F,size=(nObs,len(cols)))

    y=x[:,cols]+np.random.normal(0,sigma0*sigma1F,size=(nObs,len(cols)))
    x=np.append(x,y,axis=1)
    #3) add common random shock
    point=np.random.randint(sLength,nObs-1,size=2)
    x[np.ix_(point,[cols[0],size0])]=np.array([[-.5,-.5],[2,2]])
    #print np.ix_(point,[cols[0],size0])
    #4) add specific random shock
    point=np.random.randint(sLength,nObs-1,size=2)
    x[point,cols[-1]]=np.array([-.5,2])
    #print point,cols[-1]
    x=pd.DataFrame(x,columns=range(1,x.shape[1]+1))
    return x,cols
#------------------------------------------------------------------------------
def getHRP(cov,corr):
    # Construct a hierarchical portfolio
    corr,cov=pd.DataFrame(corr),pd.DataFrame(cov)
    #print corr
    #plotCorrMatrix('HRP3_corrAgain.png',corr)
    dist=correlDist(corr)
    link=sch.linkage(dist,'single')
    sortIx=getQuasiDiag(link)
    sortIx=corr.index[sortIx].tolist() # recover labels
    df0=corr.loc[sortIx,sortIx] # reorder
    #plotCorrMatrix('HRP3_corrAgain1.png',df0,labels=df0.columns)
    hrp=getRecBipart(cov,sortIx)
    return hrp.sort_index()

#------------------------------------------------------------------------------
def plotCorrMatrix(path,corr,labels=None):
    # Heatmap of the correlation matrix
    if labels is None:labels=[]
    mpl.pcolor(corr)
    mpl.colorbar()
    mpl.yticks(np.arange(.5,corr.shape[0]+.5),labels)
    mpl.xticks(np.arange(.5,corr.shape[0]+.5),labels)
    mpl.savefig(path)
    mpl.clf();mpl.close() # reset pylab
    return
#------------------------------------------------------------------------------

#------------------------------------------------------------------------------
def getCLA(cov,**kargs):
    # Compute CLA's minimum variance portfolio
    mean=np.arange(cov.shape[0]).reshape(-1,1) # Not used by C portf
    lB=np.zeros(mean.shape)
    uB=np.ones(mean.shape)
    cla=CLA.CLA(mean,cov,lB,uB)
    cla.solve()
    return cla.w[-1].flatten()
#------------------------------------------------------------------------------
def hrpMC(numIters=1e1,nObs=520,size0=5,size1=5,mu0=0,sigma0=1e-2, \
    sigma1F=.25,sLength=260,rebal=22):
    # Monte Carlo experiment on HRP
    methods=[getIVP,getHRP,getCLA,QPsol]
    stats,numIter={i.__name__:pd.Series() for i in methods},0
    pointers=range(sLength,nObs,rebal)
    print 'Numiters',numIters
    i=0
    name = ['IVP','HRP','CLA','QPsol']
    workbook = xlsxwriter.Workbook('WeightsSimulated.xlsx')
    worksheet = {}
    for func in methods:
        funcname = name[methods.index(func)]
        worksheet[funcname] = workbook.add_worksheet('%s'%funcname)
    while numIter<numIters:
        #print numIter
        #1) Prepare data for one experiment
        x,cols=generateData(nObs,sLength,size0,size1,mu0,sigma0,sigma1F)
        x=pd.DataFrame(x,columns=range(1,x.shape[1]+1))
        #print x.describe()
        r={i.__name__:pd.Series() for i in methods}
        #2) Compute portfolios in-sample
        for pointer in pointers:
            i=i+1
            x_=x[pointer-sLength:pointer]
            cov_,corr_,std_=np.cov(x_,rowvar=0),x_.corr(),x_.std()
            std_diag = np.diag(std_)
            corr_=corr_.values
            #plotCorrMatrix('HRP3_corr%d.png'%i,corr_)
            #3) Compute performance out-of-sample
            x_=x[pointer:pointer+rebal]
            for func in methods:
                cov_mat = cov_
                corr_mat = corr_
                w_=func(cov=cov_mat,corr=corr_mat) # callback  
                # Create a workbook and add a worksheet.
                wsheet = worksheet[name[methods.index(func)]]
                # Some data we want to write to the worksheet.
                # Start from the first cell. Rows and columns are zero indexed.
                row = 0
                col = i
                # Iterate over the data and write it out row by row.
                for item in (w_):
                    wsheet.write(row, col,     item)
                    row += 1
                r_=pd.Series(np.dot(x_,w_))
                r[func.__name__]=r[func.__name__].append(r_)
        #4) Evaluate and store results
        for func in methods:
            r_=r[func.__name__].reset_index(drop=True)
            #print 'function return',func,r_
            p_=(1+r_).cumprod()
            #print "Function and cumlative",func,p_
            stats[func.__name__].loc[numIter]=p_.iloc[-1]-1 # terminal return
        numIter+=1
    print 'Numiters',numIters
    #5) Report results
    #print stats
    stats=pd.DataFrame.from_dict(stats,orient='columns')
    stats.to_csv('stats.csv')
    df0,df1,df2=stats.std(),stats.var(),stats.mean()
    result = pd.concat([df2,df0,df2/df0,df1,df1/df1['getHRP']-1],axis=1)
    result.columns=['MEAN', 'ST. DEV.','Sharpe','VARIANCE','PERFORMANCE']
    print result
    return result
#------------------------------------------------------------------------------
if __name__=='__main__':hrpMC()

Importing ipynb file : HRP_Original.ipynb
Numiters 10.0
Numiters 10.0
            MEAN  ST. DEV.    Sharpe  VARIANCE  PERFORMANCE
QPsol   0.289796  0.332794  0.870798  0.110752     2.650919
getCLA  0.289795  0.332794  0.870794  0.110752     2.650928
getHRP  0.173662  0.174170  0.997084  0.030335     0.000000
getIVP  0.206845  0.230038  0.899180  0.052917     0.744412


## FURTHER RESEARCH
The methodology introduced in this paper is flexible, scalable and admits multiple variations of the same ideas. At stage 1 they can apply alternative definitions of linkage metrics clustering algorithms; at stage 3, they can use different functions for or alternative allocation constraints. Instead of carrying out a recursive bisection, stage 3 could also split allocations top-down using the clusters from stage 1. In a future note, we will show that it is relatively straightforward to incorporate forecasted returns and Black-Litterman-style views to this hierarchical approach. In fact, the inquisitive reader may have realized that, at its core, HRP is essentially a robust procedure to avoid matrix inversions, and the same ideas underlying HRP can be used to replace many econometric regression methods, notorious for their unstable outputs (like VAR or VECM). Exhibit 9 displays a large correlation matrix of fixed income securities before and after clustering, with over 2.1 million entries. Traditional optimization or econometric methods fail to recognize the hierarchical structure of financial Big Data, where the numerical instabilities defeat the benefits of the analysis, resulting in unreliable and detrimental outcomes.

## Simulation with some mild correlations